In [45]:
import sys

In [46]:
class Gate:
    def __init__(self, gate_type, inputs, outputs, delays, areas):
        self.gate_type = gate_type  # encoding the type of gate
        self.inputs = inputs  # inputs to the gate
        self.outputs = outputs  # outputs of the gate
        self.delay = delays  # encoding the delay
        self.area = areas # encoding the area
        self.prev = []  # to store the previous gates

In [47]:
def input():
    intput_str = ""
    input_filename = str(sys.argv[2])
    with open(input_filename, 'r') as file:
        intput_str = file.read()

    gate_delay_str = ""
    delay_filename = str(sys.argv[3])
    with open(delay_filename, 'r') as file:
        gate_delay_str = file.read()

    return intput_str, gate_delay_str

In [48]:
def formatted_input():
    input_spice, gate_delay_str = input()

    # ------------------- input_spice -----------------------
    # list that breaks the input into elements
    input_lines = input_spice.strip().split('\n')
    formatted_input_lines = []
    for line in input_lines:
        try:
            if not line[0]:
                continue
            elif line[0] == '/' and line[1] == '/':
                continue
            else:
                formatted_input_lines.append(line)
        except:
            continue

    # ------------------- delay -------------------------------
    delay_lines = gate_delay_str.strip().split('\n')
    formatted_delay_lines = []
    for line in delay_lines:
        try:
            if not line[0]:
                continue
            elif line[0] == '/' and line[1] == '/':
                continue
            else:
                formatted_delay_lines.append(line)
        except:
            continue

    return formatted_input_lines, formatted_delay_lines

In [49]:
def parse_input(lines, delays_list):
    delays_dict = {}  # dict for later use {gate_type: delay}
    area_dict = {}  # dict for later use {gate_type: area}
    delays_list = delays_list.split("\n")
    lines = lines.split("\n")
    iterator = 0
    while (iterator < len(delays_list)):
        temp_1 = delays_list[iterator].split(" ")
        temp_2 = delays_list[iterator+1].split(" ")
        temp_3 = delays_list[iterator+2].split(" ")
        gate = temp_1[1]
        delays = [float(temp_1[2]), float(temp_2[2]), float(temp_3[2])]
        areas = [float(temp_1[3]), float(temp_2[3]), float(temp_3[3])]
        delays_dict[gate] = delays
        area_dict[gate] = areas
        iterator += 3

    primary_inputs = lines[0].split()[1:]
    primary_outputs = lines[1].split()[1:]

    gates = []  # array of Gate(s)
    for line in lines[3:]:
        tokens = line.split()
        gate_type = tokens[0]
        inputs = tokens[1:-1]
        output = tokens[-1]
        if (gate_type == "DFF"):
            gates.append(Gate(gate_type,  inputs, output, [0,0,0], [0,0,0]))
        else:
            gates.append(Gate(gate_type,  inputs, output, delays_dict[gate_type], area_dict[gate_type]))

    # primary_inputs is an array of primary inputs
    # primary_outputs is an array of primary outputs
    # internal_signals is an array of internal signals
    # the array of gates
    return primary_inputs, primary_outputs, gates

In [50]:
lines = """PRIMARY_INPUTS A B
PRIMARY_OUTPUTS E F
INTERNAL_SIGNALS C D G
INV A C
AND2 C B D
DFF D G
NAND2 C G E
INV G F"""

gate_delays = """NAND2_1 NAND2 3.5 11.2
NAND2_2 NAND2 3 13
NAND2_3 NAND2 4.5 5.3
AND2_1 AND2 16.2 9.5
AND2_2 AND2 7 12
AND2_3 AND2 4 19.6
NOR2_1 NOR2 3.5 10
NOR2_2 NOR2 3 12.5
NOR2_3 NOR2 2.5 15
OR2_1 OR2 4.5 12.8
OR2_2 OR2 7.5 10.3
OR2_3 OR2 3.75 17
INV_1 INV 2 7.33
INV_2 INV 3 4.6
INV_3 INV 3.5 2.5"""


In [56]:
primary_inputs, primary_outputs, gates = parse_input(lines, gate_delays)

[2.0, 3.0, 3.5]
[16.2, 7.0, 4.0]
[0, 0, 0]
[3.5, 3.0, 4.5]
[2.0, 3.0, 3.5]


In [60]:
def max_delay(list_gate, dict):
    list_delay = []
    for gate_input in list_gate:
        list_delay.append(dict[gate_input])
    return max(list_delay)

In [57]:
dict = {}  # signal vs time delay
for signal in primary_inputs:
    dict[signal] = 0

In [69]:
to_compare = []
for gate in gates:
    if (gate.gate_type == "DFF"):
        to_compare.append(dict[gate.inputs[0]])
        dict[gate.outputs[0]] = 0
    else:
        dict[gate.outputs] = max_delay(gate.inputs, dict) + min(gate.delay)

for outputs in primary_outputs:
    to_compare.append(dict[outputs])

to_compare

[6.0, 5.0, 2.0]

In [71]:
answer = max(to_compare)
answer

6.0